In [ ]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline
import plotly.subplots

from IPython.display import HTML
from IPython.display import SVG

In [ ]:
dict_nom = {1: 'Gazole', 2: 'SP95', 3: 'E85', 4: 'GPLc', 5: 'E10', 6: 'SP98'}
dict_address = {67000002: '67000, Route de la Wantzenau', 67000003: '67000, 49 Route du Rhin', 67000004: '67000, 82 Rue du Faubourg-National', 67000005: '67000, 41 Boulevard de Lyon', 67000006: '67000, 50 Boulevard de la Victoire', 67000007: '67000, CENTRE ROUTIER DE RHEINFELD', 67000009: '67000, 10 Rue de Rome', 67000010: '67000, 169 Route de Schirmeck', 67000011: "67000, 9 Avenue d'Alsace", 67000012: '67000, 59 Rue Boecklin', 67000013: '67000, 57 Rue de Ribeauvillé', 67000015: '67000, rue saint léon', 67000016: '67000, 1 rue saint léon', 67000017: '67000, 1 rue Saint Léon', 67000018: '67000, 1 rue Saint Léon', 67000019: '67000, 1 Rue Saint-Léon', 67000020: '67000, 1 rue saint léon', 67000022: '67000, 6 Rue du Rheinfeld', 67000023: '67000, 1 rue saint léon', 67000024: '67000, TEST', 67000025: '67000, 50 BOULEVARD DE LA VICTOIRE', 67000026: '67000, 49 RTE DU RHIN', 67000027: '67000, CENTRE ROUTIER DE RHEINFELD', 67000028: '67000, 59 RUE BOECKLIN', 67100002: '67100, 250 Avenue de Colmar', 67100003: "67100, 4 Route d'Altenheim", 67100005: '67100, 212 Avenue de Colmar', 67100007: '67100, 10 Rue du Landsberg', 67100008: '67100, RUE DE RHEINFELD', 67100009: "67100, 4 ROUTE D'ALTENHEIM", 67200001: '67200, 284 ROUTE DE SCHIRMECK', 67200002: '67200, RUE  DU  MARCHE  GARE', 67200003: '67200, 68 Route de Schirmeck', 67200004: '67200, 142 Route d\'Oberhausbergen', 67200005: '67200, 48 Route des Romains', 67200006: '67200, 31 Route de Mittelhausbergen', 67200007: "67200, Rue d'Ostwald", 67200008: '67200, 31 Route de Mittelhausbergen', 67200009: '67200, 48, avenue des Romaines', 67200010: '67200, 48-58 route des romains', 67200011: '67200, 68 ROUTE DE SCHIRMECK', 67200012: "67200, 142 ROUTE D' OBERHAUSBERGEN", 67300001: '67300, Avenue Pierre Mendès France', 67300002: '67300, 90 Route du Général de Gaulle', 67300003: '67300, AV PIERRE MENDES FRANCE', 67300004: '67300, 90 ROUTE DU GENERAL DE GAULLE', 67400001: '67400, 6 Avenue de Strasbourg', 67400002: '67400, 38 Route de Lyon', 67400003: '67400, 145 Route de Lyon', 67400004: '67400, rue des vignes', 67400005: '67400, 145 ROUTE DE LYON', 67400006: '67400, 145 ROUTE DE LYON'}

In [ ]:
df = pd.read_csv('raw/prix-carburants.csv')

In [ ]:
# remove invalid cp
df = df.drop(df[df['pdv_id'] == 67033001].index)
df = df.drop(df[df['pdv_id'] == 67205001].index)

In [ ]:
# sort by seller and date
df = df.sort_values(by=["pdv_id", "date"])

In [ ]:
# date tiemstamp to datetime
df['datetime'] = pd.to_datetime(df['date'], unit='s')
# station name from pdv_id
df['station'] = df['pdv_id'].map(dict_address)

In [ ]:
for carburant_id in [1, 2, 3, 4, 5, 6]:
    df_ = df[df.carburant_id == carburant_id]
    fig = px.line(df_, x="datetime", y="carburant_prix", color='station', title=dict_nom[carburant_id])
    fig.update_layout(xaxis_range=[datetime.datetime(2012, 1, 1, 1, 1),
                                   datetime.datetime(2022, 12, 31, 1, 1)])
    fig.update_layout(yaxis_range=[0, 5])
    plotly.offline.plot(fig, filename=f'prix-carburants-{dict_nom[carburant_id]}.html', auto_open=False)
    fig.show()

In [ ]:
start = datetime.datetime(2020, 1, 1, 1, 1)
stop  = datetime.datetime(2022, 12, 31, 1, 1)

for carburant_id in [1, 6]:
    prices = []

    for pdv_id in df.pdv_id.unique():
        x = []
        y = []

        df_ = df[df.pdv_id == pdv_id]
        df_ = df_[df_.carburant_id == 1]

        for index, row in df_.iterrows():
            x.append(row['date'])
            y.append(row['carburant_prix'])

        x_ = np.linspace(start.timestamp(), stop.timestamp(), 1095)
        y_ = np.interp(x_, x, y)

        # keep if enough data
        if sum((data > start.timestamp() and data < stop.timestamp()) for data in x) < 100:
            continue
        '''
        plt.plot(x, y)
        plt.xlim(x_[0], x_[-1])
        plt.show()
        plt.plot(x_, y_)
        plt.xlim(x_[0], x_[-1])
        plt.show()
        '''

        prices.append((pdv_id, y_))
    
    prices = np.array(prices)
    means = prices[:, 1].mean()
    
    print(dict_nom[carburant_id])
    print(f'|{"Adresse":40}|{"Diff":8}|')
    print(f'|----------------------------------------|--------|')
    for i in range(len(prices)):
        diff = prices[i, 1] - means
        print(f'|{dict_address[prices[i, 0]]:40}|{round(diff.mean(), 3):8}|')